In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
def mic_equity_geog(df_in, equity_group):

    df_inc = df.pivot_table(
        index='person_work_mic',
        columns=equity_group,
        values='psexpfac',
        aggfunc='sum'
    ).reset_index()

    df_inc = df_inc.rename_axis(None, axis=1)
    pd.set_option('display.float_format', '{:,.0f}'.format)
    df_inc['Percent in Equity Geog'] = df_inc[1] / (df_inc[0] + df_inc[1])
    df_inc['Percent in Equity Geog'] = df_inc['Percent in Equity Geog'].fillna(0)
    df_inc['Percent in Equity Geog'] = (df_inc['Percent in Equity Geog'] * 100).round(1).astype(str) + '%'
    df_inc.sort_values(by=0, ascending=False)

    df_inc.rename(columns={'person_work_mic': 'MIC Work Location',
                        0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
    df_inc.index = df_inc['MIC Work Location']
    df_inc.drop('MIC Work Location', axis=1, inplace=True)
    # Percent of people of color
    
    return df_inc

## Workers in Manufacturing-Industrial Centers (MICs)

In [3]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')

In [4]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df = df[df['pwtyp'].isin(['Paid Full-Time Worker', 'Paid Part-Time Worker'])]
df.loc[df['person_work_mic'].isnull(), 'person_work_mic'] = 'Outside MIC'
_df = df[['psexpfac','person_work_mic']].groupby('person_work_mic').sum()[['psexpfac']]
_df.index
_df.rename(columns={'psexpfac': 'Total Workers'}, inplace=True)
_df


,Total Workers
person_work_mic,
Ballard-Interbay,"15,739"
Cascade,"8,765"
Duwamish,"56,586"
Frederickson,"4,094"
Kent MIC,"42,814"
North Tukwila,"6,278"
Outside MIC,"2,084,308"
Paine Field / Boeing Everett,"35,405"
Port of Tacoma,"9,809"


In [5]:
mic_equity_geog(df, "hh_efa_pov200").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"11,418","2,889","1,432",20.2%
Cascade,"5,063","2,555","1,147",33.5%
Duwamish,"33,900","14,533","8,153",30.0%
Frederickson,"2,126","1,459",509,40.7%
Kent MIC,"21,454","12,811","8,549",37.4%
North Tukwila,"3,173","1,906","1,199",37.5%
Outside MIC,"1,299,106","521,588","263,614",28.6%
Paine Field / Boeing Everett,"20,543","9,527","5,335",31.7%
Port of Tacoma,"4,463","3,284","2,062",42.4%


In [6]:
mic_equity_geog(df, "hh_efa_poc").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"10,014","3,799","1,926",27.5%
Cascade,"7,674",936,155,10.9%
Duwamish,"25,481","16,338","14,767",39.1%
Frederickson,"2,248","1,311",535,36.8%
Kent MIC,"14,991","13,049","14,774",46.5%
North Tukwila,"2,251","1,747","2,280",43.7%
Outside MIC,"1,124,761","610,030","349,517",35.2%
Paine Field / Boeing Everett,"20,028","12,285","3,092",38.0%
Port of Tacoma,"4,623","3,238","1,948",41.2%


In [7]:
mic_equity_geog(df, "hh_efa_lep").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"10,877","2,849","2,013",20.8%
Cascade,"7,383",981,401,11.7%
Duwamish,"30,009","12,617","13,960",29.6%
Frederickson,"3,188",585,321,15.5%
Kent MIC,"17,717","10,830","14,267",37.9%
North Tukwila,"2,702","1,478","2,098",35.4%
Outside MIC,"1,298,768","440,251","345,289",25.3%
Paine Field / Boeing Everett,"19,085","8,263","8,057",30.2%
Port of Tacoma,"6,455","1,958","1,396",23.3%


## Commute Characteristics

In [8]:
df = pd.read_csv(r'../../../../outputs/agg/dash/tour_mic_dest.csv')
# Commute tours to MICs
df = df[df['pdpurp']=="Work"]
df.loc[df['tour_d_mic'].isnull(), 'tour_d_mic'] = 'Outside MIC'

# tmodetp by tour_d_mic
df_mode = pd.pivot_table(
    df,
    index='tour_d_mic',
    columns='tmodetp',
    values='toexpfac',
    aggfunc='sum'
)

# mode share by tour_d_mic
df_mode = df_mode.div(df_mode.sum(axis=1), axis=0)
df_mode = df_mode.fillna(0)
df_mode = df_mode.reset_index()
df_mode.rename(columns={'tour_d_mic': 'MIC Work Location'}, inplace=True)
df_mode.index = df_mode['MIC Work Location']
df_mode.drop('MIC Work Location', axis=1, inplace=True)

df_mode['Transit'] = df_mode['Transit']+df_mode['Park']
df_mode['HOV'] = df_mode['HOV2'] + df_mode['HOV3+']
df_mode['Walk/Bike/Other'] =  df_mode['Walk']+df_mode['Bike']+df_mode['TNC']
# df_mode = df_mode.sort_values(by='Drive Alone', ascending=False)
# Show results as percentages
df_mode = df_mode.map(lambda x: f"{x:.1%}")
# df_mode.sort_values(by='Walk', ascending=False, inplace=True)


df_mode[['SOV', 'HOV', 'Transit', 'Walk/Bike/Other']]

tmodetp,SOV,HOV,Transit,Walk/Bike/Other
MIC Work Location,,,,
Ballard-Interbay,65.2%,25.7%,1.6%,7.6%
Cascade,66.9%,30.4%,0.1%,2.6%
Duwamish,65.9%,28.3%,3.0%,2.8%
Frederickson,67.9%,30.4%,0.0%,1.8%
Kent MIC,65.9%,31.2%,1.3%,1.5%
North Tukwila,66.7%,29.8%,1.4%,2.1%
Outside MIC,61.7%,27.6%,3.5%,7.2%
Paine Field / Boeing Everett,67.6%,30.1%,0.3%,2.0%
Port of Tacoma,66.8%,31.2%,0.7%,1.3%


In [9]:
# Commute distance
pd.set_option('display.float_format', '{:,.1f}'.format)

df = pd.read_csv(r'../../../../outputs/agg/dash/tour_distance_mic.csv')
df = df[df['pdpurp'] == 'Work']
df['wt_dist'] = df['tautodist_bin'] * df['toexpfac']
df = df.groupby("person_work_mic").agg(
    {'wt_dist': 'sum', 'toexpfac': 'sum'}
)
df['average_distance'] = df['wt_dist']/df['toexpfac']
df[['average_distance']]

,average_distance
person_work_mic,
Ballard-Interbay,10.3
Cascade,10.7
Duwamish,12.6
Frederickson,8.7
Kent MIC,12.6
North Tukwila,12.8
Paine Field / Boeing Everett,11.2
Port of Tacoma,11.9
Puget Sound Industrial Center- Bremerton,12.9


## Demographics

In [10]:
# Demographics 
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['income_wt'] = df['hhincome_thousands'] * df['psexpfac']
df['income_wt'] = df['income_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'income_wt': 'sum'}
).reset_index()
df["avg_weighted_hh_income"] = df['income_wt']/df['psexpfac']
df['avg_weighted_hh_income'] = df['avg_weighted_hh_income'].apply(lambda x: f"${x:,.0f}")
df[['person_work_mic', 'avg_weighted_hh_income']]

,person_work_mic,avg_weighted_hh_income
0,Ballard-Interbay,"$189,643"
1,Cascade,"$152,704"
2,Duwamish,"$185,091"
3,Frederickson,"$143,886"
4,Kent MIC,"$162,816"
5,North Tukwila,"$164,547"
6,Paine Field / Boeing Everett,"$157,121"
7,Port of Tacoma,"$145,004"
8,Puget Sound Industrial Center- Bremerton,"$144,729"
9,Sumner Pacific,"$150,495"


In [11]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['hhsize_wt'] = df['hhsize'] * df['psexpfac']
df['hhsize_wt'] = df['hhsize_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'hhsize_wt': 'sum'}
)
df['avg_wt_hhsize'] = df['hhsize_wt']/df['psexpfac']
df = df.reset_index()
df[['person_work_mic', 'avg_wt_hhsize']]

,person_work_mic,avg_wt_hhsize
0,Ballard-Interbay,2.9
1,Cascade,3.5
2,Duwamish,3.1
3,Frederickson,3.5
4,Kent MIC,3.4
5,North Tukwila,3.2
6,Paine Field / Boeing Everett,3.2
7,Port of Tacoma,3.4
8,Puget Sound Industrial Center- Bremerton,3.2
9,Sumner Pacific,3.4


In [12]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
# Pivot table of worker type by person_work_mic
df_worker_type = df.pivot_table(
    index='person_work_mic',
    columns='pwtyp',
    values='psexpfac',
    aggfunc='sum'
)

# Share by worker type

df_worker_type = df_worker_type.div(df_worker_type.sum(axis=1), axis=0)
df_worker_type.rename(columns={'Paid Full-Time Worker': 'Full Time', 'Paid Part-Time Worker': 'Part Time'})
df_worker_type.map(lambda x: f"{x:.1%}")

pwtyp,Paid Full-Time Worker,Paid Part-Time Worker
person_work_mic,,
Ballard-Interbay,82.7%,17.3%
Cascade,73.2%,26.8%
Duwamish,87.1%,12.9%
Frederickson,69.4%,30.6%
Kent MIC,85.3%,14.7%
North Tukwila,80.3%,19.7%
Paine Field / Boeing Everett,83.6%,16.4%
Port of Tacoma,84.5%,15.5%
Puget Sound Industrial Center- Bremerton,64.6%,35.4%
